In [1]:
import os
import math 
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

import mne
from mne import Epochs, pick_types, annotations_from_events, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import SPoC
from src.CSP import CSP
from mne.viz import plot_events, plot_montage
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs, corrmap, Xdawn

mne.set_log_level("CRITICAL")

# Find the best Pipeline

In [2]:
tmin, tmax = -1.0, 2.0
subject = 1
subjects = [7, 42]
drop_channels = False
crop_train = True

In [3]:
experiments = [
    {
        "runs": [3, 7, 11],
        "mapping": {0: "rest", 1: "left/fist", 2: "right/fist"},
        "event_id": {"left/fist": 1, "right/fist": 2},
    },
    {
        "runs": [4, 8, 12],
        "mapping": {0: "rest", 1: "left/imaginefist", 2: "right/imaginefist"},
        "event_id": {"left/imaginefist":1, "right/imaginefist":2},
    },
    {
        "runs": [5, 9, 13],
        "mapping": {0: "rest", 1: "top/fists", 2: "bottom/feets"},
        "event_id": {"top/fists": 1, "bottom/feets": 2},
    },
    {
        "runs": [6, 10, 14],
        "mapping": {0: "rest", 1: "top/imaginefists", 2: "top/imaginefeets"},
        "event_id": {"top/imaginefists": 1, "top/imaginefeets": 2},
    },
]
two_experiments = [
    {
        "runs": [3, 7, 11, 4, 8, 12],
        "mapping": {0: "rest", 1: "left/fist", 2: "right/fist"},
        "event_id": {"left/fist": 1, "right/fist": 2},
    },
    {
        "runs": [5, 9, 13, 6, 10, 14],
        "mapping": {0: "rest", 1: "top/fists", 2: "bottom/feets"},
        "event_id": {"top/fists": 1, "bottom/feets": 2},
    },
]
side_experiments = [
    {
        "runs": [3, 7, 11, 4, 8, 12],
        "mapping": {0: "rest", 1: "left/fist", 2: "right/fist"},
        "event_id": {"left/fist": 1, "right/fist": 2},
    },
    {
        "runs": [5, 9, 13, 6, 10, 14],
        "mapping": {0: "rest", 1: "top/fists", 2: "bottom/feets"},
        "event_id": {"top/fists": 1, "bottom/feets": 2},
    },
]

In [4]:
decoders = [
    CSP(n_components=2),
    CSP(n_components=4),
    CSP(n_components=10),
    CSP(n_components=15),
    # SPoC(n_components=2, log=True, reg='oas', rank='full'),
    # SPoC(n_components=4, log=True, reg='oas', rank='full'),
    # SPoC(n_components=10, log=True, reg='oas', rank='full'),
    # SPoC(n_components=15, log=True, reg='oas', rank='full'),
    # PCA(n_components=4),
    # PCA(n_components=8),
]
classifiers = [
    SVC(),
    LogisticRegression(penalty='l1', solver='liblinear', multi_class='auto'),
    RandomForestClassifier(n_estimators=150, random_state=42),
    LinearDiscriminantAnalysis(),
    # Ridge()
]

pipelines = []
for decomposer in decoders:
    for classifier in classifiers:
        clf = make_pipeline(decomposer, classifier)
        pipelines.append(clf)
print(f"Testing {len(pipelines)} pipelines")

Testing 16 pipelines


In [5]:
def cleanup_raw(raw):
    eegbci.standardize(raw)  # set channel names
    montage = make_standard_montage("biosemi64")
    raw.set_montage(montage, on_missing='ignore')
    
    # Select channels
    if drop_channels:
        channels = raw.info["ch_names"] 
        good_channels = [
            "FC3",
            "FC1",
            "FCz",
            "FC2",
            "FC4",
            "C3",
            "C1",
            "Cz",
            "C2",
            "C4",
            "CP3",
            "CP1",
            "CPz",
            "CP2",
            "CP4",
            "Fpz",
        ]
        bad_channels = [x for x in channels if x not in good_channels]
        raw.drop_channels(bad_channels)

    # Filter
    raw.notch_filter(60, method="iir")
    raw.filter(7.0, 32.0, fir_design="firwin", skip_by_annotation="edge") 

In [6]:
def setup_epochs_4_split():
    experiments_epochs = []
    for experiment_id, experiment in enumerate(experiments):
        raw_fnames = [f"dataset/S{subject:03d}/S{subject:03d}R{run:02d}.edf" for run in experiment["runs"]]
        raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
        events, _ = events_from_annotations(raw, event_id=dict(T0=0, T1=1, T2=2))
        annot_from_events = annotations_from_events(
            events=events, event_desc=experiment["mapping"], sfreq=raw.info["sfreq"]
        )
        raw.set_annotations(annot_from_events)
        cleanup_raw(raw)

        # Read epochs 
        events, _ = events_from_annotations(raw, event_id=experiment["event_id"])
        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
        epochs = Epochs(raw, events, experiment["event_id"], tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
        experiments_epochs.append(epochs)
    return experiments_epochs

In [7]:
def setup_epochs_2_split():
    experiments_epochs = []
    for experiment_id, experiment in enumerate(two_experiments):
        raw_fnames = [f"dataset/S{subject:03d}/S{subject:03d}R{run:02d}.edf" for run in experiment["runs"]]
        raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
        events, event_id = events_from_annotations(raw, event_id=dict(T0=0, T1=1, T2=2))
        print(event_id)
        annot_from_events = annotations_from_events(
            events=events, event_desc=experiment["mapping"], sfreq=raw.info["sfreq"]
        )
        raw.set_annotations(annot_from_events)
        cleanup_raw(raw)

        # Read epochs 
        events, _ = events_from_annotations(raw, event_id=experiment["event_id"])
        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
        epochs = Epochs(raw, events, experiment["event_id"], tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
        experiments_epochs.append(epochs)
    return experiments_epochs

In [8]:
def setup_epochs_side_merged():
    subject_raws = []
    for experiment in side_experiments:
        raw_fnames = [f"dataset/S{subject:03d}/S{subject:03d}R{run:02d}.edf" for run in experiment["runs"]]
        experiment_raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
        events, _ = events_from_annotations(experiment_raw, event_id=dict(T1=1, T2=2))
        annot_from_events = annotations_from_events(
            events=events, event_desc=experiment["mapping"], sfreq=experiment_raw.info["sfreq"]
        )
        experiment_raw.set_annotations(annot_from_events)
        subject_raws.append(experiment_raw)

    raw = concatenate_raws(subject_raws)
    cleanup_raw(raw)

    # Filter
    raw.notch_filter(60, method="iir")
    raw.filter(7.0, 32.0, fir_design="firwin", skip_by_annotation="edge") 

    # Read epochs 
    events, event_id = events_from_annotations(raw)
    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
    epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
    return [epochs]

In [9]:
def setup_epochs_all_merged():
    subject_raws = []
    for experiment in experiments:
        raw_fnames = [f"dataset/S{subject:03d}/S{subject:03d}R{run:02d}.edf" for run in experiment["runs"]]
        experiment_raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
        events, _ = events_from_annotations(experiment_raw, event_id=dict(T1=1, T2=2))
        annot_from_events = annotations_from_events(
            events=events, event_desc=experiment["mapping"], sfreq=experiment_raw.info["sfreq"]
        )
        experiment_raw.set_annotations(annot_from_events)
        subject_raws.append(experiment_raw)

    raw = concatenate_raws(subject_raws)
    cleanup_raw(raw)

    # Read epochs 
    events, event_id = events_from_annotations(raw)
    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
    epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
    return [epochs]

In [10]:
def setup_epochs_merged_subjects():
    experiments_epochs = []
    for experiment_id, experiment in enumerate(experiments):
        experiment_raws = []
        for subject in subjects:
            raw_fnames = [f"dataset/S{subject:03d}/S{subject:03d}R{run:02d}.edf" for run in experiment["runs"]]
            raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
            events, _ = events_from_annotations(raw, event_id=dict(T0=0, T1=1, T2=2))
            annot_from_events = annotations_from_events(
                events=events, event_desc=experiment["mapping"], sfreq=raw.info["sfreq"]
            )
            raw.set_annotations(annot_from_events)
            experiment_raws.append(raw)

        raw = concatenate_raws(experiment_raws)
        cleanup_raw(raw)

        # Read epochs 
        events, _ = events_from_annotations(raw, event_id=experiment["event_id"])
        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
        epochs = Epochs(raw, events, experiment["event_id"], tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
        experiments_epochs.append(epochs)
    return experiments_epochs

In [11]:
experiments_epochs = setup_epochs_4_split()
# experiments_epochs = setup_epochs_2_split()
# experiments_epochs = setup_epochs_side_merged()
# experiments_epochs = setup_epochs_all_merged()
# experiments_epochs = setup_epochs_merged_subjects()
experiments_epochs 

[<Epochs |  45 events (all good), -1 - 2 sec, baseline off, ~10.7 MB, data loaded,
  'left/fist': 23
  'right/fist': 22>,
 <Epochs |  45 events (all good), -1 - 2 sec, baseline off, ~10.7 MB, data loaded,
  'left/imaginefist': 23
  'right/imaginefist': 22>,
 <Epochs |  45 events (all good), -1 - 2 sec, baseline off, ~10.7 MB, data loaded,
  'top/fists': 23
  'bottom/feets': 22>,
 <Epochs |  45 events (all good), -1 - 2 sec, baseline off, ~10.7 MB, data loaded,
  'top/imaginefists': 21
  'top/imaginefeets': 24>]

In [12]:
all_results = []
best_model = None
for index, pipeline in enumerate(pipelines):
    print(f"# Pipeline {index + 1: 2}/{len(pipelines)}: {pipeline}")
    pipeline_accuracies = []
    pipeline_scores = [] 
    for exp_index, epochs in enumerate(experiments_epochs): 
        epochs_data = epochs.get_data()
        labels = epochs.events[:, -1]

        # monte-carlo cross-validation generator:
        cv = ShuffleSplit(10, test_size=0.2, random_state=42)

        # Display accuracy
        epochs_train = epochs.copy()
        if crop_train:
            epochs_train = epochs_train.crop(1, 2)
        score = cross_val_score(pipeline, epochs_train.get_data(), labels, cv=cv, n_jobs=-1, verbose=False).mean()
        pipeline.fit(epochs_data, labels)
        accuracy = pipeline.score(epochs_data, labels)
        print(f"[Experiment {exp_index + 1}] Accuracy: {accuracy:.2} (score: {score:.2})")
        pipeline_accuracies.append(accuracy) 
        pipeline_scores.append(score)
    accuracy = np.mean(pipeline_accuracies)
    score = np.mean(pipeline_scores)
    if best_model is None:
        best_model = (index, pipeline, accuracy, score)
    elif best_model[3] < score:
        best_model = (index, pipeline, accuracy, score)
    result = f"Accuracy {accuracy:.2} ~{np.std(pipeline_accuracies):.2} | score {score:.2} ~{np.std(pipeline_scores):.2}"
    all_results.append(result)
    print(result)
    print()

# Pipeline  1/16: Pipeline(steps=[('csp', CSP(n_components=2)), ('svc', SVC())])
[Experiment 1] Accuracy: 0.76 (score: 0.56)
[Experiment 2] Accuracy: 0.91 (score: 0.63)
[Experiment 3] Accuracy: 0.91 (score: 0.92)
[Experiment 4] Accuracy: 0.78 (score: 0.89)
Accuracy 0.84 ~0.073 | score 0.75 ~0.16

# Pipeline  2/16: Pipeline(steps=[('csp', CSP(n_components=2)),
                ('logisticregression',
                 LogisticRegression(penalty='l1', solver='liblinear'))])
[Experiment 1] Accuracy: 0.76 (score: 0.56)
[Experiment 2] Accuracy: 0.91 (score: 0.66)
[Experiment 3] Accuracy: 0.91 (score: 0.91)
[Experiment 4] Accuracy: 0.87 (score: 0.93)
Accuracy 0.86 ~0.064 | score 0.76 ~0.16

# Pipeline  3/16: Pipeline(steps=[('csp', CSP(n_components=2)),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=150, random_state=42))])
[Experiment 1] Accuracy: 1.0 (score: 0.58)
[Experiment 2] Accuracy: 1.0 (score: 0.63)
[Experiment 3] Accuracy: 1.0 (score: 0

In [13]:
for result in all_results:
    print(result)

Accuracy 0.84 ~0.073 | score 0.75 ~0.16
Accuracy 0.86 ~0.064 | score 0.76 ~0.16
Accuracy 1.0 ~0.0 | score 0.75 ~0.14
Accuracy 0.85 ~0.079 | score 0.76 ~0.13
Accuracy 0.96 ~0.029 | score 0.73 ~0.16
Accuracy 0.94 ~0.029 | score 0.75 ~0.18
Accuracy 1.0 ~0.0 | score 0.74 ~0.19
Accuracy 0.9 ~0.033 | score 0.73 ~0.18
Accuracy 0.99 ~0.019 | score 0.72 ~0.13
Accuracy 0.99 ~0.0096 | score 0.75 ~0.18
Accuracy 1.0 ~0.0 | score 0.74 ~0.17
Accuracy 0.97 ~0.036 | score 0.71 ~0.13
Accuracy 0.99 ~0.019 | score 0.72 ~0.13
Accuracy 1.0 ~0.0 | score 0.76 ~0.18
Accuracy 1.0 ~0.0 | score 0.69 ~0.1
Accuracy 1.0 ~0.0 | score 0.69 ~0.14


In [14]:
print(best_model)
best_model[1]

(1, Pipeline(steps=[('csp', CSP(n_components=2)),
                ('logisticregression',
                 LogisticRegression(penalty='l1', solver='liblinear'))]), 0.861111111111111, 0.7638888888888888)


Pipeline(steps=[('csp', CSP(n_components=2)),
                ('logisticregression',
                 LogisticRegression(penalty='l1', solver='liblinear'))])